In [2]:
import numpy as np ;
from keras.utils import np_utils
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Reshape,Dropout
from keras.preprocessing.image import ImageDataGenerator 
from keras.applications.vgg16 import VGG16
from keras.models import Model
import itertools 
import cv2
import pandas as pd ;
import matplotlib.pyplot as plt ;

train_directory  = "G1020"

#Constants 
img_rows = 224 
img_cols = 224 
inputshape = (img_rows,img_cols,3)
epochs = 2
batch_size = 5
num_of_classes = 2 
num_of_train_samples = 4983

train_generator = ImageDataGenerator().flow_from_directory(directory=train_directory , 
                                                        class_mode='categorical',
                                                        batch_size=batch_size,
                                                        target_size=(img_rows,img_cols),
                                                        color_mode="rgb",
                                                        shuffle=False);
print(train_generator.class_indices)
def getVGG16Model(lastFourTrainable = False):
    vgg_model = VGG16(weights='imagenet',input_shape=inputshape , include_top=True)
    for layer in vgg_model.layers[:]:
        layer.trainable = False
    output = vgg_model.get_layer('fc2').output
    output=Flatten(name='new_flatten' )(output) 
    output = Dense(units=128, activation='relu',name='new_fc')(output)
    output = Dense(units=2,activation='softmax')(output)
    vgg_model = Model(vgg_model.input , output)

    if lastFourTrainable == True:
        vgg_model.get_layer('block5_conv3').trainable = True
        vgg_model.get_layer('fc1').trainable = True
        vgg_model.get_layer('fc2').trainable = True
        vgg_model.get_layer('new_fc').trainable = True

    vgg_model.compile(optimizer='adam' , loss='catagorical_crossentropy',metrics=['accuracy'])
    vgg_model.summary()

    return vgg_model;

def getFeatureVector(model ,img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    feature_vector = model.predict(img.reshape(1,224,224,3))
    return feature_vector

def getFeatureDataFrame(model):
    df= pd.DataFrame(columns=['file','features'])
    train_files = train_generator.filepaths ; 
    files = train_files ; 
    df['file'] = files 
    df['features'] = df.apply(lambda row:getFeatureVector(model,row['file']),axis=1)
    return df 

    
def main():
    vgg_model_a = getVGG16Model(lastFourTrainable=False);
    feature_model_vgg_a = Model(inputs=vgg_model_a.input , outputs=vgg_model_a.get_layer('new_fc').output)
    df = getFeatureDataFrame(feature_model_vgg_a)
    df.to_pickle("G1020_VGG16.pickle")

if __name__ == "__main__":
    main();

Found 4983 images belonging to 2 classes.
{'Glaucoma': 0, 'Healthy': 1}


2023-01-28 23:01:58.466831: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

ValueError: cannot reshape array of size 200704 into shape (1,224,224,3)